# MovieLens Dataset

In this notebook, a simple example is illustrated on how to use MinHashing and LSH to find similar users based on movies they have rated. I will use one of the several datasets provided by [GroupLens](https://grouplens.org/), specifically the [MovieLens 20M Dataset](https://grouplens.org/datasets/movielens/). The dataset contains 20 million ratings applied to 27.000 movies by 138.000 users. It is 190 MB zipped, and 876 MB uncompressed. The zip folder contains several files, but we are only interested on the `ratings.csv`, which is around 533 MB.  

We can use `pandas` to read the csv file into a dataframe, and display the first 5 lines.

In [16]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')

ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


We are only intersted in two columns, namely, `userId` and `movieId`, which represent the users and movies, respectively. An important thing to note here is that these columns are encoded as integers. For example, the first user corresponds to integer 1, the second user to integer 2, etc. The same holds for `movieId`. Another way to think about it is that the values in `userId` and `movieId` are the row and column indices of a sparse term-incidence matrix, represented for example as a `scipy.sparse` matrix. 

We need to convert `userId` and `movieId` columns into a numpy array, and store it into a new variable called `data`. The new variable contains the actual data that we will use to find similar users.

In [19]:
import numpy as np

data = ratings.iloc[:, 0:2].values
data

array([[     1,      2],
       [     1,     29],
       [     1,     32],
       ..., 
       [138493,  69644],
       [138493,  70286],
       [138493,  71619]])

Finally, we need to make sure that the values of users and movies start from 0 (the first user/movie correspond to integer 0). This can be achieved easily by subtracting 1 from each row in both columns.

In [20]:
data[:, 0] = data[:, 0] - 1
data[:, 1] = data[:, 1] - 1
data

array([[     0,      1],
       [     0,     28],
       [     0,     31],
       ..., 
       [138492,  69643],
       [138492,  70285],
       [138492,  71618]])

# Input Parameters

We initialize the class using `jss.MinHashingLSH`. 

In [21]:
import JaccardSimilaritySearch as jss

mhlsh = jss.MinHashingLSH(data, 
                          threshold=0.5, 
                          n_bands=22, 
                          n_rows=6, 
                          min_set_size=10, 
                          random_state=10, 
                          verbose=1)

Parameters
----------
Nr. of Hash Functions: 132
Nr. of Bands: 22
Nr. of Rows per Band: 6
Threshold: 0.5
----------


`threshold` represents the jaccard similarity score for which, all users with similarity greater than `threshold` are considered to be similar. 

The main parameters of MinHashingLSH algorithm are the number of bands (`n_bands`) and the number of rows per band (`n_rows`). These two define the number of hash functions to use in MinHashing, where `n_hash_functions = n_bands * n_rows`. I will not get into details on what these parameters mean or how to tune them. For details you can refer to the book of [Mining of Massive Datasets](http://www.mmds.org/). Intuitively, for a fixed number of bands, increasing the number of rows per band tends to make the MinHashingLSH algorithm to finish faster, with the expense of finding less similar items. Decreasing the number of rows per band or increasing the number of bands has the opposite effect. For a fixed number of rows per band, increasing the number of bands will make the algorithm slower, but is possible to find more similar pairs of objects. Decreasing the number of bands has the opposite effect. 

`min_set_size` is a filter parameter that can be used to remove users that have rated a number of movies that is less than `min_set_size`. In other words, remove sets that contain less than `min_set_size` elements. This can be extremely useful in cases where there are many objects/sets that contain only one (or generally few) identical elements. For these sets, the corresponding columns of the signature matrix are identical, and subsequently those objects are always considered as "candidates". This means that the Jaccard similarity needs to be computed for those sets, which can be time consuming if there are many sets with one (or few) identical elements.

In [22]:
mhlsh.MinHashing(low_memory=False)


MinHashing...
Signature Matrix Created. MinHashing Completed.



In [23]:
mhlsh.LSH()

Locality Sensitive Hashing...
Processing Band: 22/22
Locality Sensitive Hashing Completed.

15170 Similar Pairs of Items Found.
